In [20]:
import requests
import urllib.request
import time
import types
from bs4 import BeautifulSoup

from datetime import datetime

In [ ]:
# Get HTML

In [2]:
url = 'https://crossfitarch.com/workout-of-the-day/'

In [3]:
response = requests.get(url)

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# Compare current week no. with week no on website

In [6]:
h1 = soup.find('h1')

In [7]:
header_week_no = h1.text.split(' ')[-1]
header_week_no = int(header_week_no)
header_week_no

25

In [9]:
current_week_no = datetime.today().isocalendar()[1]
current_week_no

25

In [78]:
header_week_no == current_week_no

True

In [ ]:
# parse HTML and read data into dictionary

In [42]:
def replace_with_newlines(element):
    text = ''
    for elem in element.recursiveChildGenerator():
        if isinstance(elem, str):
            text += elem.strip()
        elif elem.name == 'br':
            text += '\n'
        elif elem.name == 'strong':
            text += '\n'
    return text

In [77]:
days = ["montag", "dienstag", "mittwoch", "donnerstag", "freitag", "samstag", "sonntag"]
workouts = {}

for day in days:
    day_divs = soup.find("div", {"id": day})

    content = day_divs.find("div", {'class': 'elementor-text-editor elementor-clearfix'})
    full_day = replace_with_newlines(content)
    full_day = full_day.strip()
    
    wod = ''
    endurance = ''
    last_header = ''

    for line in full_day.split('\n'):
        line = line.strip()

        if line.upper() == 'WOD' or line.upper() == 'ENDURANCE' or line.upper() == 'BASIC':
            last_header = line.upper()

        else:
            if last_header == 'WOD' and len(line)>0:
                wod += line + '\n'
            elif last_header == 'ENDURANCE' and len(line)>0:
                endurance += line + '\n'
                
    workouts[day.capitalize()] = {
        'WOD': wod,
        'ENDURANCE': endurance
    }

In [90]:
mail_content = ""

for day, exercises in workouts.items():
    mail_content += day
    mail_content += '\n\n'
    
    for type_, content in exercises.items():
        if len(content) > 0:
            mail_content += (type_ + '\n')
            mail_content += (content + '\n')
        
    mail_content += '\n'

In [89]:
from recipients import recipients
from config import pw
from email.message import EmailMessage

In [88]:
msg = EmailMessage()
msg.set_content(mail_content)

# me == the sender's email address
# you == the recipient's email address
msg['Subject'] = f'Workouts KW{header_week_no} jetzt online!'
msg['From'] = "heinzeckert6@gmail.com"
msg['To'] = recipients

s = smtplib.SMTP('smtp.gmail.com', 587) 
s.starttls() 
s.login(sender, pw) 

s.send_message(msg)
s.quit()

(221, b'2.0.0 closing connection w8sm10053536eds.41 - gsmtp')